This model train on account who add more products on June 2015. Use only demographic features. 

In [9]:
import numpy as np
import pandas as pd 
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [10]:
demographic_cols = ['fecha_dato',
 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

train_cols = demographic_cols + product_col

# Import Data

In [11]:
df_train = pd.read_csv('cleaned_data/juneExtra.csv', usecols=train_cols)

/Users/rkcosmos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df_test = pd.read_csv('input/test_ver2.csv', usecols = demographic_cols)

/Users/rkcosmos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
pd.set_option('display.max_columns', None)

# Clean Data

In [14]:
df_train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-06-28,1063273,N,ES,V,37,2012-09-19,0.0,34,1.0,1.0,A,S,N,NaN,KFC,N,1.0,28.0,1.0,107894.52,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,1.0,1
1,2015-06-28,1063250,N,ES,H,40,2012-09-19,0.0,34,1.0,1.0,A,S,N,NaN,KFC,N,1.0,28.0,1.0,93847.89,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,1
2,2015-06-28,1063226,N,ES,V,46,2012-09-19,0.0,34,1.0,1.0,I,S,N,NaN,KFC,N,1.0,29.0,0.0,54195.30,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,1
3,2015-06-28,1063241,N,ES,H,41,2012-09-19,0.0,34,1.0,1.0,A,S,N,NaN,KFC,N,1.0,32.0,1.0,108502.56,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-06-28,1063292,N,ES,V,65,2012-09-19,0.0,34,1.0,1.0,A,S,N,NaN,KFC,N,1.0,29.0,1.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,1


In [15]:
df_train = df_train[df_train['ind_nuevo'] == 0]
df_train = df_train[df_train['antiguedad'] != -999999]
df_train = df_train[df_train['indrel'] == 1]
df_train = df_train[df_train['indresi'] == 'S']
df_train = df_train[df_train['indfall'] == 'N']
df_train = df_train[df_train['tipodom'] == 1]
df_train = df_train[df_train['ind_empleado'] == 'N']
df_train = df_train[df_train['pais_residencia'] == 'ES']
df_train = df_train[df_train['indrel_1mes'] == 1]
df_train = df_train[df_train['tiprel_1mes'] == ('A' or 'I')]
df_train = df_train[df_train['indext'] == 'N']

In [16]:
df_train.isnull().sum()

fecha_dato                   0
ncodpers                     0
ind_empleado                 0
pais_residencia              0
sexo                         0
age                          0
fecha_alta                   0
ind_nuevo                    0
antiguedad                   0
indrel                       0
indrel_1mes                  0
tiprel_1mes                  0
indresi                      0
indext                       0
conyuemp                 28560
canal_entrada                0
indfall                      0
tipodom                      0
cod_prov                     0
ind_actividad_cliente        0
renta                     3599
segmento                     0
ind_ahor_fin_ult1            0
ind_aval_fin_ult1            0
ind_cco_fin_ult1             0
ind_cder_fin_ult1            0
ind_cno_fin_ult1             0
ind_ctju_fin_ult1            0
ind_ctma_fin_ult1            0
ind_ctop_fin_ult1            0
ind_ctpp_fin_ult1            0
ind_deco_fin_ult1            0
ind_deme

In [17]:
df_train.tiprel_1mes.value_counts()

A    28561
Name: tiprel_1mes, dtype: int64

I will also drop 'tiprel_1mes' since we only have 'A' in train data.

### Drop unneccessary column

In [18]:
drop_column = ['ind_nuevo','indrel','indresi','indfall','tipodom','ind_empleado','pais_residencia','indrel_1mes','indext','conyuemp','fecha_alta','tiprel_1mes']

df_train.drop(drop_column, axis=1, inplace = True)
df_test.drop(drop_column, axis=1, inplace = True)

In [19]:
df_test.dtypes

fecha_dato                object
ncodpers                   int64
sexo                      object
age                        int64
antiguedad                 int64
canal_entrada             object
cod_prov                 float64
ind_actividad_cliente      int64
renta                     object
segmento                  object
dtype: object

### Add missing income

In [20]:
df_train["renta"]   = pd.to_numeric(df_train["renta"], errors="coerce")
df_test["renta"]   = pd.to_numeric(df_test["renta"], errors="coerce")

unique_prov = df_test[df_test.cod_prov.notnull()].cod_prov.unique()
grouped = df_test.groupby("cod_prov")["renta"].median()

for cod in unique_prov:
    df_test.loc[df_test['cod_prov']==cod,['renta']] = df_test.loc[df_test['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values
    df_train.loc[df_train['cod_prov']==cod,['renta']] = df_train.loc[df_train['cod_prov']==cod,['renta']].fillna({'renta':grouped[cod]}).values

In [21]:
df_test.renta.fillna(df_test["renta"].median(), inplace=True)

In [22]:
df_train.isnull().sum()

fecha_dato               0
ncodpers                 0
sexo                     0
age                      0
antiguedad               0
canal_entrada            0
cod_prov                 0
ind_actividad_cliente    0
renta                    0
segmento                 0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
ind_tjcr_fin_ult1        0
ind_valo_fin_ult1        0
ind_viv_fin_ult1         0
ind_nomina_ult1          0
ind_nom_pens_ult1        0
ind_recibo_ult1          0
dtype: int64

In [23]:
df_test.isnull().sum()

fecha_dato                  0
ncodpers                    0
sexo                        5
age                         0
antiguedad                  0
canal_entrada            2081
cod_prov                 3996
ind_actividad_cliente       0
renta                       0
segmento                 2248
dtype: int64

### Convert and make dummy

In [24]:
# not in use
replace_mapping = {
                  "sexo": {"V":0,'H':1},
                  }

df_train.replace(replace_mapping, inplace= True)

In [25]:
# These column are categories feature, I'll transform them using get_dummy
dummy_col = ['sexo','canal_entrada','cod_prov','segmento']

In [26]:
limit = int(0.05 * len(df_train.index))

for col in dummy_col:
    if len(df_train[col].unique()) > 6:
        trainlist = df_train[col].value_counts()
        use_col = []
        for i,item in enumerate(trainlist):
            if item > limit:
                use_col.append(df_train[col].value_counts().index[i])       
        for item in df_train[col].unique(): 
            if item not in use_col:
                row_index = df_train[col] == item
                df_train.loc[row_index,col] = np.nan
        for item in df_test[col].unique(): 
            if item not in use_col:
                row_index = df_test[col] == item
                df_test.loc[row_index,col] = np.nan 

In [27]:
df_train = pd.get_dummies(df_train, prefix=dummy_col, columns = dummy_col)
df_test = pd.get_dummies(df_test, prefix=dummy_col, columns = dummy_col)

In [28]:
df_train.head()

,fecha_dato,ncodpers,age,antiguedad,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,sexo_0,sexo_1,canal_entrada_KAT,canal_entrada_KFC,canal_entrada_KHE,cod_prov_8.0,cod_prov_28.0,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO
0,2015-06-28,1063273,37,34,1.0,107894.52,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,1.0,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2015-06-28,1063250,40,34,1.0,93847.89,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2015-06-28,1063241,41,34,1.0,108502.56,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2015-06-28,1063292,65,34,1.0,94965.63,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,2015-06-28,1063179,23,34,1.0,96494.64,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [29]:
df_test.head()

,fecha_dato,ncodpers,age,antiguedad,ind_actividad_cliente,renta,sexo_H,sexo_V,canal_entrada_KAT,canal_entrada_KFC,canal_entrada_KHE,cod_prov_8.0,cod_prov_28.0,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO
0,2016-06-28,15889,56,256,1,326124.90,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2016-06-28,1170544,36,34,0,67337.88,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016-06-28,1170545,22,34,1,97392.27,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2016-06-28,1170547,22,34,0,148402.98,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,2016-06-28,1170548,22,34,0,106885.80,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [30]:
df_train["age"]   = pd.to_numeric(df_train["age"], errors="coerce")
df_test["age"]   = pd.to_numeric(df_test["age"], errors="coerce")

age_group = [[0,19],[19,26],[26,36],[36,41],[41,47],[47,55],[55,60],[60,70],[70,80],[80,170]]

def create_age_group(df):  
    df['age_group'] = np.nan
    for i,age in enumerate(age_group):
        row_index = (df['age'] >= age[0]) & (df['age'] < age[1])
        df.loc[row_index,'age_group'] = i
        
create_age_group(df_train)
create_age_group(df_test)

df_train.drop('age', axis=1, inplace = True)
df_test.drop('age', axis=1, inplace = True)

In [31]:
renta_group = [[0,50000],[50000,70000],[70000,100000],[100000,150000],[150000,200000],[200000,1000000],[1000000,29000000]]

def create_renta_group(df):  
    df['renta_group'] = np.nan
    for i,renta in enumerate(renta_group):
        row_index = (df['renta'] >= renta[0]) & (df['renta'] < renta[1])
        df.loc[row_index,'renta_group'] = i
        
create_renta_group(df_train)
create_renta_group(df_test)

df_train.drop('renta', axis=1, inplace = True)
df_test.drop('renta', axis=1, inplace = True)

# Model

In [32]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = df_test['ncodpers'].values
for c in product_col:
    if c != 'ncodpers':
        print(c)
        # train model here with june 2015 data
        y_train = df_train[c]
        x_train = df_train.drop(product_col + ['ncodpers',"fecha_dato"], 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        
        # pridict model with the most recent data
        #y_train2 = df_train[c]
        x_train2 = df_test.drop(['ncodpers',"fecha_dato"], 1)
        p_train = clf.predict_proba(x_train2)[:,1]
        p_train2 = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train
        
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train2))
        

ind_ahor_fin_ult1
0.916943870584
ind_aval_fin_ult1
0.828670308378
ind_cco_fin_ult1
0.589738343227
ind_cder_fin_ult1
0.811196924733
ind_cno_fin_ult1
0.612721006316
ind_ctju_fin_ult1
0.99946402298
ind_ctma_fin_ult1
0.749300813989
ind_ctop_fin_ult1
0.807033913909
ind_ctpp_fin_ult1
0.711205981961
ind_deco_fin_ult1
0.730691335952
ind_deme_fin_ult1
0.738464001648
ind_dela_fin_ult1
0.78077668253
ind_ecue_fin_ult1
0.671674653116
ind_fond_fin_ult1
0.809511639389
ind_hip_fin_ult1
0.813341060198
ind_plan_fin_ult1
0.780151907566
ind_pres_fin_ult1
0.775178918299
ind_reca_fin_ult1
0.669902836744
ind_tjcr_fin_ult1
0.694399391696
ind_valo_fin_ult1
0.79391574608
ind_viv_fin_ult1
0.829893836704
ind_nomina_ult1
0.638732296182
ind_nom_pens_ult1
0.623026838435
ind_recibo_ult1
0.606282516806


In [ ]:
df_train.shape

# Make submission

In [ ]:
df_recent = pd.read_csv('input/train_ver2.csv',usecols=['ncodpers'] + product_col)
df_recent = df_recent.drop_duplicates(['ncodpers'], keep='last')

In [ ]:
sample = pd.read_csv('input/sample_submission.csv')

In [ ]:
# check if customer already have each product or not. 
already_active = {}
for row in df_recent.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(tuple(product_col), row) if c[1] > 0]
    already_active[id] = active

# add 7 products(that user don't have yet), higher probability first -> train_pred   
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(tuple(product_col), p) if i[0] not in already_active[id]],
                                  key=lambda i:i [1], 
                                  reverse=True)[:7]]
    train_preds[id] = preds
    
test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))


In [ ]:
sample['added_products'] = test_preds
sample.to_csv('output/LR_demographic2.csv', index=False)